<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inititial-Record-Ingestion" data-toc-modified-id="Inititial-Record-Ingestion-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inititial Record Ingestion</a></span><ul class="toc-item"><li><span><a href="#Connect-To-And-Re-Create-DB" data-toc-modified-id="Connect-To-And-Re-Create-DB-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Connect To And Re-Create DB</a></span></li><li><span><a href="#Read-Back-Up-Files" data-toc-modified-id="Read-Back-Up-Files-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Read Back-Up Files</a></span></li><li><span><a href="#Write-Records-To-DB" data-toc-modified-id="Write-Records-To-DB-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Write Records To DB</a></span></li><li><span><a href="#Check-The-Results" data-toc-modified-id="Check-The-Results-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Check The Results</a></span></li></ul></li><li><span><a href="#Initial-Credit-Addition" data-toc-modified-id="Initial-Credit-Addition-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Initial Credit Addition</a></span></li></ul></div>

In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy import func

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration2 import Base, Artist, CreditTrx, Genre, Label, Rating, Record
from src import db_functions
from src import db_connect

In [4]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Inititial Record Ingestion

In [5]:
CONFIG_PATH = Path.cwd().parent / "config.yaml"

### Connect To And Re-Create DB

<div class="alert alert-block alert-danger">
Attention This Will Delete All Entries In The DB!
</div>

In [6]:
prod_path = r"sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\DiscoBase2.db"
# dev_path = r"sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\dev\DeafDiscoBase.db"


engine = sqlalchemy.create_engine(prod_path)
session = db_connect.create_session(engine)

# Check
engine
session

Engine(sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\DiscoBase2.db)

In [7]:
# Re-Create
# db_functions.create_DB_anew(engine, Base)

### Read Back-Up Files

In [8]:
def load_backup_data_from_parquet(
    config_path: Union[Path, str], 
    record_data_name: Union[Path, str],
    trx_data_name: Union[Path, str]
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load the two back-up files with the record-related and 
    the credit_trx data into Pandas DataFrames.
    """
    
    back_up_params = db_connect.read_yaml(config_path, "BACK_UP")
    rel_path = back_up_params["REL_PATH"]
    target_folder = Path.cwd().parent / rel_path  # TODO CHANGE FOR PROD
    
    df_list = []
    for file in [record_data_name, trx_data_name]:
        full_path = target_folder / file
    
        df = pd.read_parquet(full_path)
        df_list.append(df)
        
    return df_list[0], df_list[1]

In [9]:
record_data, trx_data = load_backup_data_from_parquet(
    CONFIG_PATH, 
    "record_data_2021-02-14-15-58-43.parquet",
    "trx_data_2021-02-14-15-58-44.parquet"
)

In [10]:
record_data.info()
record_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308 entries, 1 to 308
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   artist          308 non-null    object        
 1   artist_country  13 non-null     object        
 2   title           308 non-null    object        
 3   genre           308 non-null    object        
 4   label           308 non-null    object        
 5   year            308 non-null    int64         
 6   record_format   308 non-null    object        
 7   vinyl_color     212 non-null    object        
 8   lim_edition     222 non-null    object        
 9   number          24 non-null     object        
 10  remarks         78 non-null     object        
 11  price           308 non-null    float64       
 12  purchase_date   308 non-null    datetime64[ns]
 13  rating          103 non-null    float64       
 14  is_digitized    308 non-null    bool          
 15  is_act

,artist,artist_country,title,genre,label,year,record_format,vinyl_color,lim_edition,number,remarks,price,purchase_date,rating,is_digitized,is_active
record_id,,,,,,,,,,,,,,,,
1,Dismember,None,Pieces,Death Metal,[NA],1992,"12""",None,None,None,None,35.00,1992-01-01,nan,False,True
2,Dismember,None,Skin Her Alive,Death Metal,[NA],1991,"7"" Pic",None,None,None,None,15.00,1992-01-01,nan,False,True
3,Dismember,None,Like An Everflowing Stream,Death Metal,[NA],1991,LP,None,None,None,None,155.00,1992-01-01,nan,False,True
4,Dismember,None,Where Ironcrosses Flow,Death Metal,[NA],2004,LP,None,None,None,None,15.00,1992-01-01,nan,True,True
5,Merciless,None,The Awakening,Death Metal,[NA],1989,LP,None,None,None,"New, Wert geschätzt / 1. der beiden Originalpressungen auf DSP",250.00,1992-01-01,nan,True,True


In [11]:
EDA.display_nan(record_data)

,total,prop,dtype
artist_country,295,95.8%,object
number,284,92.2%,object
remarks,230,74.7%,object
rating,205,66.6%,float64
vinyl_color,96,31.2%,object
lim_edition,86,27.9%,object


In [12]:
# splits_plus_one = record_data[record_data["artist"].str.contains("/")].index
# splits_plus_one

In [13]:
# TEMPORARY: Bring Artists To List

record_data["artist_country"] = record_data["artist_country"].fillna("NA")

def split_strings(x):
    if isinstance(x, str):
        x = x.split(" / ")
    return x

record_data.loc[288, "artist_country"] = "USA / Australia"
record_data["artist"] = record_data["artist"].apply(split_strings)
record_data["artist_country"] = record_data["artist_country"].apply(split_strings)

# record_data[record_data.index.isin(splits_plus_one)]

In [14]:
record_data.to_dict("records")[0]
type(record_data.iloc[0,0][0])

{'artist': ['Dismember'],
 'artist_country': ['NA'],
 'title': 'Pieces',
 'genre': 'Death Metal',
 'label': array(['NA'], dtype=object),
 'year': 1992,
 'record_format': '12"',
 'vinyl_color': None,
 'lim_edition': None,
 'number': None,
 'remarks': None,
 'price': 35.0,
 'purchase_date': Timestamp('1992-01-01 00:00:00'),
 'rating': nan,
 'is_digitized': False,
 'is_active': True}

str

**TEMPORARY TO DOS:**

- fillna in Ratings col because it will generate an FK in the future

In [15]:
# todo

In [16]:
# SIMULATE A TRX

record_data["credit_value"] = 0
record_data["trx_type"] = "Initial Load"

### Write Records To DB

**TODO:** Can I apply()?

In [17]:
def insert_df_with_sqlalchemy_orm(session, df):
    for x in df.to_dict("records"):
        db_functions.add_new_record(session, x)


In [18]:
insert_df_with_sqlalchemy_orm(session, record_data)

C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


Record 'Pieces' by ['Dismember'] already exists, insert skipped.
['Dismember']
Record 'Skin Her Alive' by ['Dismember'] already exists, insert skipped.
['Dismember']
Record 'Like An Everflowing Stream' by ['Dismember'] already exists, insert skipped.
['Dismember']
Record 'Where Ironcrosses Flow' by ['Dismember'] already exists, insert skipped.
['Dismember']
Record 'The Awakening' by ['Merciless'] already exists, insert skipped.
['Merciless']
Record 'Ironbound' by ['Overkill'] already exists, insert skipped.
['Overkill']
Record 'The Years Of Decay' by ['Overkill'] already exists, insert skipped.
['Overkill']
Record 'Horrorscope' by ['Overkill'] already exists, insert skipped.
['Overkill']
Record 'Electric Age' by ['Overkill'] already exists, insert skipped.
['Overkill']
Record 'Live From Oz' by ['Overkill'] already exists, insert skipped.
['Overkill']
Record 'Uprising' by ['Entombed'] already exists, insert skipped.
['Entombed']
Record 'Morning Star' by ['Entombed'] already exists, inse

Record 'Instinto Ruim' by ['Worst '] already exists, insert skipped.
['Worst ']
Record 'I'm Alone Even With You' by ['Rotten Mind'] already exists, insert skipped.
['Rotten Mind']
Record 'Los Primeros 7"' by ['Suicidas '] already exists, insert skipped.
['Suicidas ']
Record 'Svartalvheim' by ['Ancient'] already exists, insert skipped.
['Ancient']
Record 'The Ten Commandments' by ['Malevolent Creation'] already exists, insert skipped.
['Malevolent Creation']
Record 'Legion' by ['Deicide'] already exists, insert skipped.
['Deicide']
Record 'Dawn Of Possession' by ['Immolation'] already exists, insert skipped.
['Immolation']
Record 'Köp Dig Lycklig' by ['Ursut'] already exists, insert skipped.
['Ursut']
Record 'Dårarnas Paradis' by ['Ursut'] already exists, insert skipped.
['Ursut']
Record 'Elddop' by ['Martyrdöd'] already exists, insert skipped.
['Martyrdöd']
Record 'Traits of The Past' by ['Agnosy'] already exists, insert skipped.
['Agnosy']
Record 'Past The Point Of No Return' by ['Agn

Record 'Cenotaph of Defectuous Creation' by ['Sempiternal Dusk'] already exists, insert skipped.
['Sempiternal Dusk']
Record 'Lost In Life' by ['Backtrack'] already exists, insert skipped.
['Backtrack']
Record 'The Wrong Way' by ['Rotting Out'] already exists, insert skipped.
['Rotting Out']
Record 'Upon The Throne of Apocalypse' by ['Incantation'] already exists, insert skipped.
['Incantation']
Record 'Odious Descent Into Decay' by ['Cerebral Rot'] already exists, insert skipped.
['Cerebral Rot']
Record 'Sonoran Depravation' by ['Gatecreeper'] already exists, insert skipped.
['Gatecreeper']
Record 'Tropical Sun' by ['Fulci'] already exists, insert skipped.
['Fulci']
Record 'From Wisdom To Baked' by ['Cannabis Corpse'] already exists, insert skipped.
['Cannabis Corpse']
Record 'Deserted' by ['Gatecreeper'] already exists, insert skipped.
['Gatecreeper']
Record 'Unhumanized' by ['Cianide'] already exists, insert skipped.
['Cianide']
Record 'Hidden History Of The Human Race' by ['Blood I

### Check The Results

In [20]:
# Total Number Of Records In The DB
session.query(Record).count()

308

In [21]:
session.query(Artist).count()
session.query(Artist).all()[-5:]

199

[<Artist(artist_id=195, artist_name=Celestial Sanctuary)>,
 <Artist(artist_id=196, artist_name=200 STAB WOUNDS)>,
 <Artist(artist_id=197, artist_name=Unurnment)>,
 <Artist(artist_id=198, artist_name=Infestment)>,
 <Artist(artist_id=199, artist_name=Heads For The Dead)>]

In [22]:
session.query(Genre).count()
session.query(Genre).all()

9

[<Genre(genre_id=1, genre_name=Death Metal)>,
 <Genre(genre_id=2, genre_name=Thrash Metal)>,
 <Genre(genre_id=3, genre_name=Black Metal)>,
 <Genre(genre_id=4, genre_name=Speed Metal)>,
 <Genre(genre_id=5, genre_name=Hardcore)>,
 <Genre(genre_id=6, genre_name=Punk)>,
 <Genre(genre_id=7, genre_name=Crust)>,
 <Genre(genre_id=8, genre_name=Crossover)>,
 <Genre(genre_id=9, genre_name=Grindcore)>]

In [30]:
session.query(CreditTrx).count()
session.query(func.sum(CreditTrx.credit_value)).all()[0][0]
session.query(func.min(CreditTrx.credit_value)).all()[0][0]
session.query(func.avg(CreditTrx.credit_value)).all()[0][0]

313

5.0

0.0

0.01597444089456869

## Initial Credit Addition

In [24]:
# Add initial Addition trx, check the date an credit_saldo

fake_trx = CreditTrx(
    credit_trx_date=dt.datetime(year=2020, month=12, day=30),
    credit_trx_type="Addition",
    credit_value=1,
    credit_saldo=-1,
    record_id=np.nan
)
session.add(fake_trx)

In [25]:
# Check
session.commit()
session.query(CreditTrx).all()[-3:]

[<CreditTrx(credit_trx_id=307, credit_trx_date=2021-02-05, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=307)>,
 <CreditTrx(credit_trx_id=308, credit_trx_date=2021-02-08, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=308)>,
 <CreditTrx(credit_trx_id=309, credit_trx_date=2020-12-30, credit_trx_type=Addition, credit_value=1.0, credit_saldo=-1.0, record_id=None)>]

In [26]:
db_functions.add_regular_credits(session)

Creating 'Addition' Trx for: 2021-01-09
Creating 'Addition' Trx for: 2021-01-19
Creating 'Addition' Trx for: 2021-01-29
Creating 'Addition' Trx for: 2021-02-08


In [27]:
session.query(CreditTrx).all()[-3:]

[<CreditTrx(credit_trx_id=311, credit_trx_date=2021-01-19, credit_trx_type=Addition, credit_value=1.0, credit_saldo=1.0, record_id=None)>,
 <CreditTrx(credit_trx_id=312, credit_trx_date=2021-01-29, credit_trx_type=Addition, credit_value=1.0, credit_saldo=2.0, record_id=None)>,
 <CreditTrx(credit_trx_id=313, credit_trx_date=2021-02-08, credit_trx_type=Addition, credit_value=1.0, credit_saldo=3.0, record_id=None)>]

In [28]:
# session.rollback()

In [29]:
session.close()